# Tests historisation


In [1]:
import os
from datetime import datetime, timedelta
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine, types, dialects
from create_query import to_indicator

engine = create_engine(os.getenv("DATABASE_URL"))

Plusieurs solutions sont testées (pour chaque niveau d'historisation):

- solution 1 : une table avec les résultats des indicateurs (plusieurs lignes par indicateur)
- solution 1bis : une table avec un champ JSON pour les valeurs (plusieurs lignes par indicateur)
- solution 2 : une table avec un champ JSON pour l'ensemble des résultats (une ligne par indicateur)

## Principes indicateurs

- résultat instantané : valeur unique (ex. puissance, nombre de stations...)
- résultat sur une période : ensemble de valeurs (ex. puissance quotidienne...)

### Historisation : Pour ne pas stocker un ensemble des valeurs on stocke des données agrégées 'scalables'

- quantité de valeurs (nécessaire pour les calculs)
- valeur moyenne
- dernière valeur de la période (optionnel ou obligatoire ?)
- écart-type (optionnel)
- valeur mini (optionnel)
- valeur maxi (optionnel)

ex. 'dernière valeur' utile quand on veut comparer une évolution entre deux dates (ex entre fin 2023 et fin 2024) ou avoir une courbe des valeurs mensuelles

### Types d'indicateur

- valeur unique (tableau avec une colonne et une valeur)
- avec un paramètre -> liste de valeurs (tableau avec deux colonnes et n valeurs)
- avec deux paramètres -> liste de valeurs (tableau avec trois colonnes et n x n valeurs)


## Tests de performance

Comparaison des temps de réponse d'une query pour les trois solutions avec le scénario suivant :

- requête t8 annuelle à partir des résultats des requêtes quotidiennes stockées sur un an dans une table.
- avec t8 : nb station par opérateur et par région-01/département-02/EPCI-03/communes-04

In [2]:
dd = timedelta(days=1)
ti = datetime.fromisoformat('2024-01-01')
test = 't8---01'
test = 't8---02'
test = 't8---03'
test = 't8---04'

### Tests perf Solution 1

In [3]:
to_indicator(engine, test, histo=True, format='table', histo_timest= ti.isoformat(), table_name='quotidien_1', table_option='replace')
for i in range(365):
    ti += dd
    to_indicator(engine, test, histo=True, format='table', histo_timest= ti.isoformat(), table_name='quotidien_1', table_option='append')

In [4]:
with engine.connect() as conn:
    quotidien_1 = pd.read_sql_query("SELECT * FROM quotidien_1", conn)
quotidien_1

,quantity,mean,last,crit_v,code,query,level,val,area,timest
0,1,266.0,266.0,IZIVIA,86233,t8,00,00,04,2024-01-01
1,1,133.0,133.0,IZIVIA,92026,t8,00,00,04,2024-01-01
2,1,100.0,100.0,IZIVIA,33073,t8,00,00,04,2024-01-01
3,1,90.0,90.0,IZIVIA,92044,t8,00,00,04,2024-01-01
4,1,84.0,84.0,UBITRICITY GMBH,76351,t8,00,00,04,2024-01-01
...,...,...,...,...,...,...,...,...,...,...
1445329,1,1.0,1.0,Bouygues Energies et Services,77512,t8,00,00,04,2024-12-31
1445330,1,1.0,1.0,Last Mile Solutions,76341,t8,00,00,04,2024-12-31
1445331,1,1.0,1.0,IONITY,62239,t8,00,00,04,2024-12-31
1445332,1,1.0,1.0,RossiniEnergy,52292,t8,00,00,04,2024-12-31


In [5]:
# passage de la table 'quotidien' à la table 'mensuel' (chaque mois on retrouve un seul enregistrement de chaque indicateur)
# le traitement pourrait se faire directement en SQL sans passer par un DataFrame
# la requête est la même pour passer de jour à mois que pour passer de mois à année

query = """
SELECT
  SUM(quantity) AS quantity,  SUM(quantity * mean) AS somme,  LAST(last) AS last, crit_v,  code, query,  level,  val,  area
FROM
  quotidien_1
WHERE
  (timest >= to_timestamp('2024-01-01', 'YYYY-MM-DD')   AND   timest < to_timestamp('2025-01-01', 'YYYY-MM-DD'))
GROUP BY
  crit_v,  code, query,  level,  val,  area
ORDER BY
  query,  level,  val,  area
"""

with engine.connect() as conn:
    mensuel = pd.read_sql_query(query, conn)
mensuel

,quantity,somme,last,crit_v,code,query,level,val,area
0,366.0,366.0,1.0,ANYOS,71448,t8,00,00,04
1,366.0,366.0,1.0,ATRACHATA,2B009,t8,00,00,04
2,366.0,366.0,1.0,AVIA LOCATION,95280,t8,00,00,04
3,366.0,366.0,1.0,Acelec Charge,59512,t8,00,00,04
4,366.0,366.0,1.0,Acelec Charge,77018,t8,00,00,04
...,...,...,...,...,...,...,...,...,...
3944,366.0,30744.0,84.0,UBITRICITY GMBH,76351,t8,00,00,04
3945,366.0,32940.0,90.0,IZIVIA,92044,t8,00,00,04
3946,366.0,36600.0,100.0,IZIVIA,33073,t8,00,00,04
3947,366.0,48678.0,133.0,IZIVIA,92026,t8,00,00,04


In [6]:
%%timeit
with engine.connect() as conn:
    mensuel = pd.read_sql_query(query, conn)

895 ms ± 72.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
mensuel.to_sql('mensuel_1', engine, if_exists='replace', index=False)

-1

### tests perf sol 1 bis

In [8]:
dtype_1_bis = {'crit_v': types.TEXT, 'code': types.TEXT, 'query': types.TEXT, 'level': types.TEXT, 'val': types.TEXT, 'area': types.TEXT, 
        'timestamp': types.TIMESTAMP, 'value': dialects.postgresql.JSONB}

In [9]:
ti = datetime.fromisoformat('2024-01-01')
to_indicator(engine, test, histo=True, format='table', histo_timest= ti.isoformat(), table_name='quotidien_1_bis', table_option='replace', table_dtype=dtype_1_bis, test='1bis')
for i in range(365):
    ti += dd
    to_indicator(engine, test, histo=True, format='table', histo_timest= ti.isoformat(), table_name='quotidien_1_bis', table_option='append', table_dtype=dtype_1_bis, test='1bis')

In [10]:
with engine.connect() as conn:
    quotidien_1_bis = pd.read_sql_query("SELECT * FROM quotidien_1_bis", conn)
quotidien_1_bis

,crit_v,code,query,level,val,area,timest,value
0,IZIVIA,86233,t8,00,00,04,2024-01-01,"{'last': 266.0, 'mean': 266.0, 'quantity': 1}"
1,IZIVIA,92026,t8,00,00,04,2024-01-01,"{'last': 133.0, 'mean': 133.0, 'quantity': 1}"
2,IZIVIA,33073,t8,00,00,04,2024-01-01,"{'last': 100.0, 'mean': 100.0, 'quantity': 1}"
3,IZIVIA,92044,t8,00,00,04,2024-01-01,"{'last': 90.0, 'mean': 90.0, 'quantity': 1}"
4,UBITRICITY GMBH,76351,t8,00,00,04,2024-01-01,"{'last': 84.0, 'mean': 84.0, 'quantity': 1}"
...,...,...,...,...,...,...,...,...
1445329,Bouygues Energies et Services,77512,t8,00,00,04,2024-12-31,"{'last': 1.0, 'mean': 1.0, 'quantity': 1}"
1445330,Last Mile Solutions,76341,t8,00,00,04,2024-12-31,"{'last': 1.0, 'mean': 1.0, 'quantity': 1}"
1445331,IONITY,62239,t8,00,00,04,2024-12-31,"{'last': 1.0, 'mean': 1.0, 'quantity': 1}"
1445332,RossiniEnergy,52292,t8,00,00,04,2024-12-31,"{'last': 1.0, 'mean': 1.0, 'quantity': 1}"


In [11]:
# passage de la table 'quotidien' à la table 'mensuel' (chaque mois on retrouve un seul enregistrement de chaque indicateur)
# le traitement pourrait se faire directement en SQL sans passer par un DataFrame
# la requête est la même pour passer de jour à mois que pour passer de mois à année

query = """
SELECT
  SUM((value->>'quantity')::float) AS quantity,  SUM((value->>'quantity')::float * (value->>'mean')::float) AS somme,  (value->>'last')::float AS last, crit_v,  code, query,  level,  val,  area
FROM
  quotidien_1_bis
WHERE
  (timest >= to_timestamp('2024-01-01', 'YYYY-MM-DD')   AND   timest < to_timestamp('2025-01-01', 'YYYY-MM-DD'))
GROUP BY
  last, crit_v,  code, query,  level,  val,  area
ORDER BY
  somme, query,  level,  val,  area
"""

with engine.connect() as conn:
    mensuel = pd.read_sql_query(query, conn)
mensuel

,quantity,somme,last,crit_v,code,query,level,val,area
0,366.0,366.0,1.0,ANYOS,71448,t8,00,00,04
1,366.0,366.0,1.0,ATRACHATA,2B009,t8,00,00,04
2,366.0,366.0,1.0,AVIA LOCATION,95280,t8,00,00,04
3,366.0,366.0,1.0,Acelec Charge,59512,t8,00,00,04
4,366.0,366.0,1.0,Acelec Charge,77018,t8,00,00,04
...,...,...,...,...,...,...,...,...,...
3944,366.0,30744.0,84.0,UBITRICITY GMBH,76351,t8,00,00,04
3945,366.0,32940.0,90.0,IZIVIA,92044,t8,00,00,04
3946,366.0,36600.0,100.0,IZIVIA,33073,t8,00,00,04
3947,366.0,48678.0,133.0,IZIVIA,92026,t8,00,00,04


In [12]:
%%timeit
with engine.connect() as conn:
    mensuel = pd.read_sql_query(query, conn)

1.31 s ± 39.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
mensuel.to_sql('mensuel_1_bis', engine, if_exists='replace', index=False)

-1

### Tests perf sol 2 

In [14]:
with engine.connect() as conn:
    quotidien_1_bis = pd.read_sql_query("SELECT * FROM quotidien_1_bis", conn)

In [15]:
quotidien_2 = quotidien_1_bis[['query', 'level', 'val', 'area', 'timest']].drop_duplicates().reset_index(drop=True)
quotidien_2['timest'] = quotidien_2['timest'].astype('string')

In [16]:
def value_2(row, df):
    param = row.to_dict()
    query = f"query == '{param['query']}' and level == '{param['level']}' and val == '{param['val']}' and area == '{param['area']}' and timest == '{param['timest']}'"
    return df.query(query)[['code', 'crit_v', 'value']].to_dict(orient='records')    

In [17]:
quotidien_1_bis['timest'] = quotidien_1_bis['timest'].astype('string')
quotidien_2['result'] = pd.Series([value_2(row, quotidien_1_bis) for row_idx, row in quotidien_2.iterrows()])
quotidien_2

,query,level,val,area,timest,result
0,t8,00,00,04,2024-01-01,"[{'code': '86233', 'crit_v': 'IZIVIA', 'value'..."
1,t8,00,00,04,2024-01-02,"[{'code': '75056', 'crit_v': 'Bump', 'value': ..."
2,t8,00,00,04,2024-01-03,"[{'code': '76165', 'crit_v': 'VIRTA', 'value':..."
3,t8,00,00,04,2024-01-04,"[{'code': '86233', 'crit_v': 'IZIVIA', 'value'..."
4,t8,00,00,04,2024-01-05,"[{'code': '35068', 'crit_v': 'Spie', 'value': ..."
...,...,...,...,...,...,...
361,t8,00,00,04,2024-12-27,"[{'code': '83019', 'crit_v': 'Bump', 'value': ..."
362,t8,00,00,04,2024-12-28,"[{'code': '52399', 'crit_v': 'Tesla', 'value':..."
363,t8,00,00,04,2024-12-29,"[{'code': '50592', 'crit_v': None, 'value': {'..."
364,t8,00,00,04,2024-12-30,"[{'code': '85127', 'crit_v': 'Bump', 'value': ..."


In [18]:
dtype_2 = {'result': dialects.postgresql.JSONB, 'query': types.TEXT, 'level': types.TEXT, 'val': types.TEXT, 'area': types.TEXT, 'timestamp': types.TIMESTAMP}
quotidien_2.to_sql("quotidien_2", engine, if_exists="replace", dtype=dtype_2)

-1

In [19]:
# passage de la table 'quotidien' à la table 'mensuel' (chaque mois on retrouve un seul enregistrement de chaque indicateur)
# le traitement pourrait se faire directement en SQL sans passer par un DataFrame
# la requête est la même pour passer de jour à mois que pour passer de mois à année

query = """
WITH quotidien_flat AS
    (SELECT 
        query, level, val, area, 
        jsonb_path_query(result, '$.code') AS code, 
        jsonb_path_query(result, '$.crit_v') AS crit_v, 
        jsonb_path_query(result, '$.value') AS value, 
        timest
    FROM 
        quotidien_2)
SELECT
  SUM((value->>'quantity')::float) AS quantity,  SUM((value->>'quantity')::float * (value->>'mean')::float) AS somme,  (value->>'last')::float AS last, crit_v,  code, query,  level,  val,  area
FROM
  quotidien_flat
WHERE
  (timest::TIMESTAMP >= to_timestamp('2024-01-01', 'YYYY-MM-DD')   AND   timest::TIMESTAMP < to_timestamp('2025-01-01', 'YYYY-MM-DD'))
GROUP BY
  last, crit_v,  code, query,  level,  val,  area
ORDER BY
  somme, query,  level,  val,  area
"""

with engine.connect() as conn:
    mensuel = pd.read_sql_query(query, conn)
mensuel

,quantity,somme,last,crit_v,code,query,level,val,area
0,366.0,366.0,1.0,Last Mile Solutions,21231,t8,00,00,04
1,366.0,366.0,1.0,Power Dot France,30189,t8,00,00,04
2,366.0,366.0,1.0,SPIE CITYNETWORKS,44120,t8,00,00,04
3,366.0,366.0,1.0,STATIONS-E,83121,t8,00,00,04
4,366.0,366.0,1.0,E-TOTEM,59646,t8,00,00,04
...,...,...,...,...,...,...,...,...,...
3944,366.0,30744.0,84.0,UBITRICITY GMBH,76351,t8,00,00,04
3945,366.0,32940.0,90.0,IZIVIA,92044,t8,00,00,04
3946,366.0,36600.0,100.0,IZIVIA,33073,t8,00,00,04
3947,366.0,48678.0,133.0,IZIVIA,92026,t8,00,00,04


In [20]:
%%timeit
with engine.connect() as conn:
    mensuel = pd.read_sql_query(query, conn)

5.97 s ± 65.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
mensuel.to_sql('mensuel_2', engine, if_exists='replace', index=False)

-1

## Résultats des tests 

Comparaison des temps de réponse d'une query pour les trois solutions avec le scénario suivant :

- requête t8 de calcul des résultats annuels à partir des résultats des requêtes quotidiennes stockées sur un an dans une table.
- avec t8 : nb station par opérateur et par région-01/département-02/EPCI-03/communes-04

| requete | s1 durée | s1bis durée |  s1 rows  | s2 durée | s2 rows |s2/s1bis|
| ------- | -------- | ----------- | --------- | -------- | ------- | ------ |
| t8---01 | 251      | 207         | 121146    | 434      | 366     |  1.8   |
| t8---02 | 286      | 440         | 389058    | 1390     | 366     |  3.2   |
| t8---03 | 497      | 690         | 822036    | 3130     | 366     |  4.5   |
| t8---04 | 861      | 1240        | 1445334   | 5980     | 366     |  4.8   |



# Annexe : autres tests

## Tests sol 1 autres

In [28]:
# ajout d'indicateurs dans la table 'quotidien'
# on simule l'envoi quotidien des indicateurs (en répétant le même indicateur)
to_indicator(engine, 'i1',          histo=True, format='table', table_name='quotidien_1', table_option='replace')
to_indicator(engine, 'i1-01-93-02', histo=True, format='table', table_name='quotidien_1', table_option='append')
to_indicator(engine, 't1-02-75',    histo=True, format='table', table_name='quotidien_1', table_option='append')
to_indicator(engine, 't3-04-13001', histo=True, format='table', table_name='quotidien_1', table_option='append')
to_indicator(engine, 'i1',          histo=True, format='table', table_name='quotidien_1', table_option='append')
to_indicator(engine, 't1-02-75',    histo=True, format='table', table_name='quotidien_1', table_option='append')
to_indicator(engine, 't3-04-13001', histo=True, format='table', table_name='quotidien_1', table_option='append')
to_indicator(engine, 't1-02-75',    histo=True, format='table', table_name='quotidien_1', table_option='append')

,count
0,33


In [29]:
# tous les indicateurs ont le même format
with engine.connect() as conn:
    quotidien_1 = pd.read_sql_table('quotidien_1', conn)
quotidien_1

,quantity,mean,last,crit_v,code,query,level,val,area,timest
0,1,23522.0,23522.0,,,i1,00,00,00,2024-09-15 21:13:52.758505+00:00
1,1,473.0,473.0,,13,i1,01,93,02,2024-09-15 21:13:52.850220+00:00
2,1,450.0,450.0,,06,i1,01,93,02,2024-09-15 21:13:52.850220+00:00
3,1,175.0,175.0,,83,i1,01,93,02,2024-09-15 21:13:52.850220+00:00
4,1,170.0,170.0,,84,i1,01,93,02,2024-09-15 21:13:52.850220+00:00
5,1,105.0,105.0,,04,i1,01,93,02,2024-09-15 21:13:52.850220+00:00
6,1,57.0,57.0,,05,i1,01,93,02,2024-09-15 21:13:52.850220+00:00
7,1,73.0,73.0,"[15.0,26.0)",,t1,02,75,00,2024-09-15 21:13:52.918944+00:00
8,1,13.0,13.0,"[26,65.0)",,t1,02,75,00,2024-09-15 21:13:52.918944+00:00
9,1,9.0,9.0,"[0,15.0)",,t1,02,75,00,2024-09-15 21:13:52.918944+00:00


In [34]:
# passage de la table 'quotidien' à la table 'mensuel' (chaque mois on retrouve un seul enregistrement de chaque indicateur)
# le traitement pourrait se faire directement en SQL sans passer par un DataFrame
# la requête est la même pour passer de jour à mois que pour passer de mois à année

query = """
SELECT
  SUM(quantity) AS quantity,  SUM(quantity * mean) / SUM(quantity) AS mean,  last, crit_v,  code, query,  level,  val,  area
FROM
  quotidien_1
WHERE
  (timest >= CAST(NOW() AS date))   AND   (timest < CAST((NOW() + INTERVAL '1 month') AS date))
GROUP BY
  last, crit_v,  code, query,  level,  val,  area
ORDER BY
  query,  level,  val,  area
"""

with engine.connect() as conn:
    mensuel = pd.read_sql_query(query, conn)
mensuel

,quantity,mean,last,crit_v,code,query,level,val,area
0,2.0,23522.0,23522.0,,,i1,00,00,00
1,1.0,57.0,57.0,,05,i1,01,93,02
2,1.0,105.0,105.0,,04,i1,01,93,02
3,1.0,170.0,170.0,,84,i1,01,93,02
4,1.0,175.0,175.0,,83,i1,01,93,02
5,1.0,450.0,450.0,,06,i1,01,93,02
6,1.0,473.0,473.0,,13,i1,01,93,02
7,3.0,3.0,3.0,"[175,360.0)",,t1,02,75,00
8,3.0,4.0,4.0,"[65,175.0)",,t1,02,75,00
9,3.0,9.0,9.0,"[0,15.0)",,t1,02,75,00


In [35]:
mensuel.to_sql('mensuel_1', engine, if_exists='replace', index=False)

-1

In [36]:
# indicateur 't1-m-02-75' sur le mois
# la requête est générique (en séparant les cas ou c'est la moyenne de ceux ou c'est la somme)

query = """
SELECT
  mean::int AS nb_pdc,  crit_v AS p_range
FROM
  mensuel_1
WHERE
  query = 't1' AND level = '02' AND val = '75'
"""
with engine.connect() as conn:
    t1_m_02_75 = pd.read_sql_query(query, conn)

# on retrouve l'indicateur du jour
t1_m_02_75

,nb_pdc,p_range
0,3,"[175,360.0)"
1,4,"[65,175.0)"
2,9,"[0,15.0)"
3,13,"[26,65.0)"
4,73,"[15.0,26.0)"


In [37]:
to_indicator(engine, 't1-02-75')

,nb_pdc,p_range
0,73,"[15.0, 26.0)"
1,13,"[26, 65.0)"
2,9,"[0, 15.0)"
3,4,"[65, 175.0)"
4,3,"[175, 360.0)"


## Tests solution 2 bis

In [38]:
import json

In [39]:
val_t8_01_93_02 = [
    {
        "department": "01",
        "operators": [
            {
                "name": "Bouygues",
                "sum": 42,
                "delta": 3
            },
            {
                "name": "Electra",
                "sum": 12,
                "delta": -2
            },
            {
                "name": "Engie",
                "sum": 252,
                "delta": 154
            }
        ]
    },
    {
        "department": "02",
        "operators": [
            {
                "name": "Bouygues",
                "sum": 4,
                "delta": 3
            },
            {
                "name": "Electra",
                "sum": 52,
                "delta": 23
            },
            {
                "name": "Engie",
                "sum": 1,
                "delta": 1
            }
        ]
    }
]

In [40]:
t8_01_93_02 = pd.DataFrame({'code': ['t8']*3, 'level': [1]*3, 'target': [93]*3, 'sub_level': [2]*3, 
                            'value': [val_t8_01_93_02]*3, 
                            'timestamp': ['2024-09-09T15:05:28+00:00', '2024-09-10T15:05:28+00:00', 
                                          '2024-09-11T15:05:28+00:00']})
t8_01_93_02

,code,level,target,sub_level,value,timestamp
0,t8,1,93,2,"[{'department': '01', 'operators': [{'name': '...",2024-09-09T15:05:28+00:00
1,t8,1,93,2,"[{'department': '01', 'operators': [{'name': '...",2024-09-10T15:05:28+00:00
2,t8,1,93,2,"[{'department': '01', 'operators': [{'name': '...",2024-09-11T15:05:28+00:00


In [41]:
dtype={'code': types.TEXT, 'level': types.SMALLINT, 'target': types.TEXT, 'sub_level': types.SMALLINT, 
       'value': dialects.postgresql.JSONB, 'timestamp': types.TIMESTAMP}
t8_01_93_02.to_sql("mensuel_3", engine, if_exists="replace", dtype=dtype)

-1

In [42]:
query = """
with dpt_ext as
    (select 
        target, level, code, sub_level, 
        jsonb_path_query(value, '$.department') as department, 
        jsonb_path_query(value, '$.operators') as oper_json, 
        timestamp
    from 
        mensuel_3)
select 
    target, level, code, sub_level, department, 
    jsonb_path_query(oper_json, '$.name') as operator, 
    jsonb_path_query(oper_json, '$.sum') as sum, 
    jsonb_path_query(oper_json, '$.delta') as delta, 
    timestamp
from 
    dpt_ext
"""
with engine.connect() as conn:
    test_json = pd.read_sql_query(query, conn)
test_json

,target,level,code,sub_level,department,operator,sum,delta,timestamp
0,93,1,t8,2,01,Bouygues,42,3,2024-09-09 15:05:28
1,93,1,t8,2,01,Electra,12,-2,2024-09-09 15:05:28
2,93,1,t8,2,01,Engie,252,154,2024-09-09 15:05:28
3,93,1,t8,2,02,Bouygues,4,3,2024-09-09 15:05:28
4,93,1,t8,2,02,Electra,52,23,2024-09-09 15:05:28
5,93,1,t8,2,02,Engie,1,1,2024-09-09 15:05:28
6,93,1,t8,2,01,Bouygues,42,3,2024-09-10 15:05:28
7,93,1,t8,2,01,Electra,12,-2,2024-09-10 15:05:28
8,93,1,t8,2,01,Engie,252,154,2024-09-10 15:05:28
9,93,1,t8,2,02,Bouygues,4,3,2024-09-10 15:05:28


In [43]:
val_t8_01_93_02 = [
    {   "code": "01",
        "crit_v": "Bouygues",
        "value": {
            'quantite': 5, 
            "moyenne": 10}},
    {   "department": "01",
        "operators": "Electra",
        "value": {
            'quantite': 15, 
            "moyenne": 20}},
    {   "department": "01",
        "operators": "Engie",
        "value": {
            'quantite': 25, 
            "moyenne": 30}},
    
    {   "department": "02",
        "operators": "Bouygues",
        "value": {
            'quantite': 5, 
            "moyenne": 10}},
    {   "department": "02",
        "operators": "Electra",
        "value": {
            'quantite': 15, 
            "moyenne": 20}},
    {   "department": "02",
        "operators": "Engie",
        "value": {
            'quantite': 25, 
            "moyenne": 30}}
]